In [ ]:
#Hacemos los imports
import pandas as pd
import numpy as np
import json
import math
from unidecode import unidecode
import statistics
import seaborn as sns
import matplotlib.pyplot as plt
import random
import statistics


In [ ]:
#Cargamos directamente lo que nos interesa, la parte 'data' del json.
with open('train-v2.0.json','r') as f:
    data = json.load(f)['data']

In [ ]:
#Normalizamos para que los datos puedan ser legibles en un data frame. Renombramos las columnas. 
data_pre= pd.json_normalize(data, record_path= ['paragraphs', 'qas', 'answers'], meta= [['title'],['paragraphs', 'context'], ['paragraphs', 'qas', 'question'], ['paragraphs', 'qas', 'is_impossible']])
data_pre.rename(columns= {'text':'answer', 'paragraphs.context':'context', 'paragraphs.qas.question':'question'}, inplace=True)
data_pre.head(3)

In [ ]:
#Lower
data_pre['answer']=data_pre['answer'].apply(str.lower)
data_pre['context']=data_pre['context'].apply(str.lower)
data_pre['question']=data_pre['question'].apply(str.lower)

In [ ]:
#Eliminamos los stopwords, acentos y carácteres especiales
stopwords=['I', 'me', 'my', 'myself', 'we','our','ours','ourselves','you','your','yours','yourself','yourselves','he','him','his','himself','she','her','hers','herself','it','its','itself','they','them','their','theirs','themselves','what','which','who','whom','this','that','these','those','am','is','are','was','were','be','been','being','have','has','had','having','do','does','did','doing','a','an','the','and','but','if','or','because','as','until','while','of','at','by','for','with','about','against','between','into','through','during','before','after','above','below','to','from','up','down','in','out','on','off','over','under','again','further','then','once','here','there','when','where','why','how','all','any','both','each','few','more','most','other','some','such','no','nor','not','only','own','same','so','than','too','very','s','t','can','will','just','don','should','now']

#hacemos el set de stop words para eliminar
sw=r'\b(?:{})\b'.format('|'.join(stopwords))
data_pre['question']=data_pre['question'].str.replace(sw, '')
data_pre['context']=data_pre['context'].str.replace(sw, '')
data_pre['answer']=data_pre['answer'].str.replace(sw, '')

data_pre['context']=data_pre['context'].str.replace(r'[?!,\'()]+', '')
data_pre['answer']=data_pre['answer'].str.replace(r'[?!,\'()]+', '')
data_pre['question']=data_pre['question'].str.replace(r'[?!,\'()]+', '')

data_pre['context']=data_pre['context'].apply(lambda x:unidecode(x))
data_pre['answer']=data_pre['answer'].apply(lambda x:unidecode(x))
data_pre['question']=data_pre['question'].apply(lambda x:unidecode(x))

#data_pre['question']=data_pre['question'].str.replace('?', '')

#data_pre['answer']=data_pre['answer'].str.replace('\W', '', regex=True)
#data_pre['question']=data_pre['question'].str.replace('\W', '', regex=True)
#data_pre['context']=data_pre['context'].str.replace('\W', '', regex=True)

In [ ]:
#Spliteamos a una lista cada oracion de context, con delimitador .
data_pre['context']=data_pre['context'].str.split(".")
#Usamos .explode para que cada oración quede con sus respectivos índices y columnas (combinaciones)
data_pre=data_pre.explode('context')

In [ ]:
#Drop empty elements
#nan=null of pandas

data_pre['context'].replace('', np.nan, inplace=True)
data_pre['question'].replace('', np.nan, inplace=True)
data_pre['answer'].replace('', np.nan, inplace=True)

data_pre.dropna(subset=["context"], inplace=True)
data_pre.dropna(subset=["question"], inplace=True)
data_pre.dropna(subset=["answer"], inplace=True)

In [ ]:
data_pre.head(10)

In [ ]:
def jaccard(list1, list2):
    intersection = len(list(set(list1).intersection(list2)))
    union = (len(list1) + len(list2)) - intersection
    return float(intersection) / union

In [ ]:
def ans(list1, list2):
    check = all(item in list1 for item in list2)
    if check is True:
        return 1;
    else:
        return 0;

In [ ]:
#Tokenizamos y hacemos el dataframe data_jaccard que tiene Ans y la distancia
#Pasamos todo a listas
data_pre['context']=data_pre['context'].str.split()
data_pre['question']=data_pre['question'].str.split()
data_pre['answer']=data_pre['answer'].str.split()

data_jaccard= pd.DataFrame()

data_jaccard["Jaccard(context_i_line_j, q_i_k)"] = data_pre.apply(lambda x: jaccard(x['context'], x['question']), axis=1)
data_jaccard["Ans"]= data_pre.apply(lambda x: ans(x['context'], x['answer']), axis=1 )

In [ ]:
data_jaccard.head(50)

In [ ]:
data_jaccard.info()

In [ ]:
#5% Hacemos el sample
sample_1_jaccard=data_jaccard.sample(n=math.ceil(490330*0.05), replace=False, random_state= np.random.seed(123454321))

#1% Hacemos el sample
sample_2_jaccard=data_jaccard.sample(n=math.ceil(490330*0.01), replace=False, random_state= np.random.seed(123454321))

In [ ]:
class Logit:
      '''
      This class is a logit classifier
      X in [m, n+1]
       n: variables
       m: observations
      y in [m, 1]
      theta in [1, n+1]
      np.dot(X, theta.T)
      
      ''' 
      def __init__(self, X, y, alpha=.005):
            self.X = self.add_ordinate(X)
            self.y = y
            self.theta = np.random.rand(X.shape[1]+1).reshape(1, -1)
            self.alpha = alpha # This is the learning rate
            self.loss_hist = []
            print(f'Loading data: X shape [{self.X.shape}]')
            print(f'Loading data: y shape [{self.y.shape}]')
            print(f'params shape: theta [{self.theta.shape}]')

      def add_ordinate(self, X):
            return np.hstack([np.ones(X.shape[0]).reshape(-1, 1), X])
      
            
      def forward(self, X=None):
            '''
            This function implements:
            the logit pass to X. 1/(1 + e-z*theta)
            '''
            X = self.add_ordinate(X) if not X is None else self.X
            print(X.shape)
            return 1/(1 + np.exp(-np.dot(X, self.theta.T)))

      def loss(self):
            '''
            Computes cross entropy loss
            '''
            p = self.forward()
            return -np.mean(self.y*np.log(p) + (1-self.y)*np.log(1-p))

      def train(self, tol=1e-5, max_iter=10000):
            iters = 0
            loss = np.Inf
            print(iters)
            while(loss > tol and iters < max_iter):
                  loss = self.loss()
                  if not iters % 500:
                        print(f'loss: {loss}')
                  p = self.forward().reshape(-1, 1)
                  self.theta -= -self.alpha*np.mean((self.y - p)*self.X, axis=0)
                  iters += 1
                  self.loss_hist.append(loss)

In [ ]:
#5% Shapeamos a arreglos
x_sample_1= sample_1_jaccard.loc[:, 'Jaccard(context_i_line_j, q_i_k)'].values.reshape(-1,1)
y_sample_1= sample_1_jaccard.loc[:, 'Ans'].values.reshape(24517,1)


In [ ]:
#1% Shapeamos a arreglos
x_sample_2= sample_2_jaccard.loc[:, 'Jaccard(context_i_line_j, q_i_k)'].values.reshape(-1,1)
y_sample_2= sample_2_jaccard.loc[:, 'Ans'].values.reshape(4904,1)

In [ ]:
#5% Clase
log_1= Logit(X=x_sample_1, y=y_sample_1)

In [ ]:
#1% Clase
log_2=Logit(X=x_sample_2, y=y_sample_2)

In [ ]:
#5% Entrenamos
log_1.train()

In [ ]:
#1%
log_2.train()

In [ ]:
#5% Obtenemos Theta del sample
sample_theta_1=getattr(log_1, 'theta')
#sample_theta=log_1.theta
print(sample_theta_1)

In [ ]:
#1% Obtenemos Theta del sample
sample_theta_2=getattr(log_2, 'theta')
#sample_theta=log_1.theta
print(sample_theta_2)

In [ ]:
#Bootstrap function that gets sample with replacement from the original smaple

def boot5(sample):
    resample= sample.sample(n=len(sample), replace=True)
    x_resample= resample.loc[:, 'Jaccard(context_i_line_j, q_i_k)'].values.reshape(-1,1)
    y_resample= resample.loc[:, 'Ans'].values.reshape(24517,1)
    log_resample= Logit(X=x_resample, y=y_resample)
    log_resample.train()
    thetas=log_resample.theta
    return thetas

In [ ]:
#Bootstrap function that gets sample with replacement from the original smaple

def boot1(sample):
    resample= sample.sample(n=len(sample), replace=True)
    x_resample= resample.loc[:, 'Jaccard(context_i_line_j, q_i_k)'].values.reshape(-1,1)
    y_resample= resample.loc[:, 'Ans'].values.reshape(4904,1)
    log_resample= Logit(X=x_resample, y=y_resample)
    log_resample.train()
    thetas=log_resample.theta
    return thetas

In [ ]:
#Creating a list to capture the boot function outputs for 5%
thetas_0_5=[]
thetas_1_5=[]

#5%
for i in range(100):
    thetas_5=boot5(sample_1_jaccard)
    thetas_0_5.append(thetas_5[0][0])
    thetas_1_5.append(thetas_5[0][1])

In [ ]:
#Creating a list to capture the boot function outputs for 1%
thetas_0_1=[]
thetas_1_1=[]

#5%
for i in range(100):
    thetas_1=boot1(sample_2_jaccard)
    thetas_0_1.append(thetas_1[0][0])
    thetas_1_1.append(thetas_1[0][1])

In [ ]:
print(thetas_0_5)
print(thetas_1_5)

print(thetas_0_1)
print(thetas_1_1)

In [ ]:
print(len(thetas_0_5))
print(len(thetas_1_5))

print(len(thetas_0_1))
print(len(thetas_1_1))


In [ ]:
import statistics
#Thetas_0 for 5%
plt.hist(thetas_0_5, bins=100, color='c', edgecolor='k')
ci=np.percentile(thetas_0, [2.5, 97.5])
lower_bound_thetas_0= ci[0]
upper_bound_thetas_0=ci[1]

plt.axvline(lower_bound_thetas_0, color='k', linestyle='dashed')
plt.axvline(upper_bound_thetas_0, color='k', linestyle='dashed')




In [ ]:
#Thetas_1 for 5%
plt.hist(thetas_1_5, bins=100, color='c', edgecolor='k')
ci=np.percentile(thetas_1, [2.5, 97.5])
lower_bound_thetas_1= ci[0]
upper_bound_thetas_1=ci[1]

plt.axvline(lower_bound_thetas_1, color='k', linestyle='dashed')
plt.axvline(upper_bound_thetas_1, color='k', linestyle='dashed')

In [ ]:
#Thetas_0 for 1%
plt.hist(thetas_0_1, bins=100, color='c', edgecolor='k')
ci=np.percentile(thetas_0_1, [2.5, 97.5])
lower_bound_thetas_0= ci[0]
upper_bound_thetas_0=ci[1]

plt.axvline(lower_bound_thetas_0, color='k', linestyle='dashed')
plt.axvline(upper_bound_thetas_0, color='k', linestyle='dashed')


In [ ]:
#Thetas_1 for 1%
plt.hist(thetas_1_1, bins=100, color='c', edgecolor='k')
ci=np.percentile(thetas_1_1, [2.5, 97.5])
lower_bound_thetas_1= ci[0]
upper_bound_thetas_1=ci[1]

plt.axvline(lower_bound_thetas_1, color='k', linestyle='dashed')
plt.axvline(upper_bound_thetas_1, color='k', linestyle='dashed')

In [ ]:
#5% Theta 1
thetas_1_5.sort()
np.percentile(thetas_1_5, [2.5, 97.5])

In [ ]:
#1% Theta 1
thetas_1_1.sort()
np.percentile(thetas_1_1, [2.5, 97.5])

In [ ]:
#5% Theta 0
thetas_0_5.sort()
np.percentile(thetas_0_5, [2.5, 97.5])

In [ ]:
#1% Theta 0
thetas_0_1.sort()
np.percentile(thetas_0_1, [2.5, 97.5])

In [ ]:
#5% Con el método Standard Error - aplicando Teorema central del límite
m1= statistics.mean(thetas_0_5)
m2= statistics.mean(thetas_1_5)

s0=statistics.stdev(thetas_0_5)
s1=statistics.stdev(thetas_1_5)

percentile_lower_thetas_0= sample_theta_1[0][0] - 2*s0
percentile_upper_thetas_0=sample_theta_1[0][0] + 2*s0

percentile_lower_thetas_1= sample_theta_1[0][1] - 2*s1
percentile_upper_thetas_1=sample_theta_1[0][1] + 2*s1

print(percentile_lower_thetas_0)
print(percentile_upper_thetas_0)
print(percentile_lower_thetas_1)
print(percentile_upper_thetas_1)

In [ ]:
#1% - TCL
m1= statistics.mean(thetas_0_1)
m2= statistics.mean(thetas_1_1)

s0=statistics.stdev(thetas_0_1)
s1=statistics.stdev(thetas_1_1)

percentile_lower_thetas_0= sample_theta_2[0][0] - 2*s0
percentile_upper_thetas_0=sample_theta_2[0][0] + 2*s0

percentile_lower_thetas_1= sample_theta_2[0][1] - 2*s1
percentile_upper_thetas_1=sample_theta_2[0][1] + 2*s1

print(percentile_lower_thetas_0)
print(percentile_upper_thetas_0)
print(percentile_lower_thetas_1)
print(percentile_upper_thetas_1)